In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
import torch.nn as nn
import torch.optim as optim
from sklearn.decomposition import PCA
from torch_geometric.utils import remove_self_loops, degree

for dim in [4, 8, 16, 32, 64]:
    print(dim)
    dataset = Planetoid(root='/tmp/cora', name="cora")
    data = dataset[0]
    # 设定降维后的目标维度
    if dim == "inf":
        pca_dim = data.x.shape[1]  # 你可以根据需要调整PCA降维后的维度'
    else:
        # 对节点特征进行PCA降维
        pca_dim = dim 
        def apply_pca(features, pca_dim):
            pca = PCA(n_components=pca_dim)
            pca_result = pca.fit_transform(features)
            return torch.tensor(pca_result, dtype=torch.float)
        
        # 将数据中的节点特征进行PCA降维
        data.x = apply_pca(data.x.numpy(), pca_dim)
    
    def gcn_conv(h, edge_index):
        N = h.size(0)
        edge_index, _ = remove_self_loops(edge_index)
        
        src, dst = edge_index
        deg = degree(dst, num_nodes=N)
        
        deg_src = deg[src].pow(-0.5) 
        deg_src.masked_fill_(deg_src == float('inf'), 0)
        deg_dst = deg[dst].pow(-0.5)
        deg_dst.masked_fill_(deg_dst == float('inf'), 0)
        edge_weight = deg_src * deg_dst
    
        a = torch.sparse_coo_tensor(edge_index, edge_weight, torch.Size([N, N])).t()
        h_prime = a @ h 
        return h_prime
    
    # 2. Define a simple neural network for estimating W
    class SimpleNN(nn.Module):
        def __init__(self, in_features, out_features):
            super(SimpleNN, self).__init__()
            self.fc = nn.Linear(in_features, out_features, bias=False)
        
        def forward(self, x):
            return self.fc(x)
    
    x = data.x
    edge_index = data.edge_index
    
    # 4. Run GCN convolution to get x'
    x_prime = gcn_conv(x, edge_index)
    # x_prime = gcn_conv(x_prime, edge_index)
    x_prime = x_prime - x
    
    # 5. Prepare data for training
    input_features = x.size(1)
    output_features = x_prime.size(1)
    model = SimpleNN(input_features, output_features)
    
    # Use Mean Squared Error as the loss function
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.1)
    
    # 6. Train the model with original x'
    num_epochs = 300
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        x_prime_pred = model(x)
        loss = criterion(x_prime_pred, x_prime)
        loss.backward()
        optimizer.step()
    
    # 7. Compute the total error Q for the original x'
    model.eval()
    with torch.no_grad():
        x_prime_pred = model(x)
        errors = torch.norm(x_prime_pred - x_prime, p=2, dim=1)
        Q_original = loss
    
    # 8. Shuffle rows of x'
    x_prime_shuffled = x_prime[torch.randperm(x_prime.size(0))]
    
    # 9. Train the model with shuffled x'
    model = SimpleNN(input_features, output_features)  # Re-initialize model
    optimizer = optim.Adam(model.parameters(), lr=0.1)
    
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        x_prime_pred_shuffled = model(x)
        loss = criterion(x_prime_pred_shuffled, x_prime_shuffled)
        loss.backward()
        optimizer.step()
    
    # 10. Compute the total error Q for the shuffled x'
    model.eval()
    with torch.no_grad():
        x_prime_pred_shuffled = model(x)
        errors_shuffled = torch.norm(x_prime_pred_shuffled - x_prime_shuffled, p=2, dim=1)
        Q_shuffled = loss
    
    print((Q_shuffled-Q_original)/Q_shuffled)

4
tensor(0.5764, grad_fn=<DivBackward0>)
8
tensor(0.5958, grad_fn=<DivBackward0>)
16
tensor(0.6241, grad_fn=<DivBackward0>)
32
tensor(0.6558, grad_fn=<DivBackward0>)
64
tensor(0.6830, grad_fn=<DivBackward0>)
